# 作業目標: 使用python正規表達式對資料進行清洗處理

這份作業我們會使用詐欺郵件的文本資料來作為清洗與處理的操作。
[資料集](https://www.kaggle.com/rtatman/fraudulent-email-corpus/data#)

### 讀入資料文本
因原始文本較大，先使用部份擷取的**sample_emails.txt**來進行練習

In [1]:
#讀取文本資料
#<your code>#

In [2]:
import pandas as pd
dataset = pd.read_table(r'sample_emails.txt',sep='\n')
print(dataset)
str = open('sample_emails.txt', 'r').read()

                      From r  Wed Oct 30 21:41:56 2002
0           Return-Path: <james_ngola2002@maktoob.com>
1                               X-Sieve: cmu-sieve 2.0
2           Return-Path: <james_ngola2002@maktoob.com>
3      Message-Id: <200210310241.g9V2fNm6028281@cs.CU>
4    From: "MR. JAMES NGOLA." <james_ngola2002@makt...
..                                                 ...
133  Nevertheless if you are for any reason not int...
134  kindly inform me immediately so that I will lo...
135             I am waiting for your quick response. 
136                                 Yours faithfully, 
137                                Prince Obong Abbot 

[138 rows x 1 columns]


### 讀取寄件者資訊
觀察文本資料可以發現, 寄件者資訊都符合以下格式

`From: <收件者姓名> <收件者電子郵件>`

In [3]:
#<your code>#

In [4]:
import re
pattern="From: \".*"
match1 = re.findall(pattern, str)
print (match1)
match1='\n'.join(match1)

['From: "MR. JAMES NGOLA." <james_ngola2002@maktoob.com>', 'From: "Mr. Ben Suleman" <bensul2004nng@spinfinder.com>', 'From: "PRINCE OBONG ELEME" <obong_715@epatra.com>']


### 只讀取寄件者姓名

In [5]:
#<your code>#

pattern="(?<=From: \")(.+)(?=\")"
match=re.findall(pattern,match1)
print(match)

['MR. JAMES NGOLA.', 'Mr. Ben Suleman', 'PRINCE OBONG ELEME']


### 只讀取寄件者電子信箱

In [6]:
pattern="(?<=\<)(.+)(?=\>)"
match=re.findall(pattern,match1)
print(match)

['james_ngola2002@maktoob.com', 'bensul2004nng@spinfinder.com', 'obong_715@epatra.com']


### 只讀取電子信箱中的寄件機構資訊
ex: james_ngola2002@maktoob.com --> 取maktoob

In [7]:
pattern="(?<=\@)(.+)(?=\.)"
match=re.findall(pattern,match1)
print(match)

['maktoob', 'spinfinder', 'epatra']


### 結合上面的配對方式, 將寄件者的帳號與機構訊返回
ex: james_ngola2002@maktoob.com --> [james_ngola2002, maktoob]

In [8]:
pattern="(?<=\<)(.+)(?=\@)|(?<=\@)(.*)(?=\.)"
match=re.findall(pattern,match1)
print(match)

[('james_ngola2002', ''), ('', 'maktoob'), ('bensul2004nng', ''), ('', 'spinfinder'), ('obong_715', ''), ('', 'epatra')]


### 使用正規表達式對email資料進行處理
這裡我們會使用到python其他的套件協助處理(ex: pandas, email, etc)，這裡我們只需要專注在正規表達式上即可，其他的套件是方便我們整理與處理資料。

### 讀取與切分Email
讀入的email為一個長字串，利用正規表達式切割讀入的資料成一封一封的email，並將結果以list表示。

輸出: [email_1, email_2, email_3, ....]

In [9]:
import re
import pandas as pd
import email

###讀取文本資料:fradulent_emails.txt###
with open('all_emails.txt', 'r',encoding="utf8", errors='ignore') as f:
    all_emails=f.read()
    
###切割讀入的資料成一封一封的email###
###我們可以使用list來儲存每一封email###
###注意！這裡請仔細觀察sample資料，看資料是如何切分不同email###
emails = re.split(r"From r", all_emails, flags=re.M)
emails = emails[1:] #移除第一項的空元素
len(emails) #查看有多少封email

3977

### 從文本中擷取所有寄件者與收件者的姓名和地址

In [12]:
emails_list = [] #創建空list來儲存所有email資訊

for mail in emails[:20]: #只取前20筆資料 (處理速度比較快)
    emails_dict = dict() #創建空字典儲存資訊
    ###取的寄件者姓名與地址###
    
    #Step1: 取的寄件者資訊 (hint: From:)
    sender = re.search(r"From:.*", mail)
    
    #Step2: 取的姓名與地址 (hint: 要注意有時會有沒取到配對的情況)
    if sender is not None: #有取到配對
        sender_mail = re.search(r"\w\S*@.*\b", sender.group())
        sender_name = re.search(r"(?<=\").*(?=\")", sender.group())
    else: #沒取到配對
        sender_mail = None
        sender_name = None

    #Step3: 將取得的姓名與地址存入字典中
    if sender_mail is not None:
        emails_dict["sender_email"] = sender_mail.group()
    else:
        emails_dict["sender_email"] = sender_mail 
    
    if sender_name is not None:
        emails_dict["sender_name"] = sender_name.group()
    else:
        emails_dict["sender_name"] = sender_name 
        
    
    ###取的收件者姓名與地址###
    #Step1: 取的收件者資訊 (hint: To:)
    recipient=re.search(r"To:.*", mail)

    
    #Step2: 取的姓名與地址 (hint: 要注意有時會有沒取到配對的情況)
    if recipient is not None:
        recipient_email = re.search(r"\w\S*@.*\b", recipient.group())
        recipient_name = re.search(r"(?<=\").*(?=\")", recipient.group())
    else:
        recipient_email = None
        recipient_name = None
        
    #Step3: 將取得的姓名與地址存入字典中
    if recipient_email is not None:
        emails_dict["recipient_mails"]=recipient_email.group()    
    else:
        emails_dict["recipient_mails"]=recipient_email 
    if recipient_name is not None:
        emails_dict["recipient_names"]=recipient_name.group() 
    else:
        emails_dict["recipient_names"]=recipient_name
        
    ###取得信件日期###
    #Step1: 取得日期資訊 (hint: To:)
    date=re.search(r"To:.*",mail)
    
    #Step2: 取得詳細日期(只需取得DD MMM YYYY)
    if date is not None:
        date_info = re.search(r"\d+\s\w+\s\d+", date.group())
    else:
        date_info = None

    #Step3: 將取得的日期資訊存入字典中
    if date_info is not None:
        emails_dict["date"]=date_info.group()
    else:
        emails_dict["date"]=date_info

        
    ###取得信件主旨###
    #Step1: 取得主旨資訊 (hint: Subject:)
    subject_info = re.search(r"Subject: .*", mail)

    
    #Step2: 移除不必要文字 (hint: Subject: )
    if subject_info is not None:
        subject=re.sub(r"Subject: ", "", subject_info.group())
    else:
        subject=None

    
    #Step3: 將取得的主旨存入字典中
    if subject is not None:
        emails_dict["emails_body"]=subject


    
    ###取得信件內文###
    #這裡我們使用email package來取出email內文 (可以不需深究，本章節重點在正規表達式)

    try:
        full_email = email.message_from_string(mail)
        body = full_email.get_payload()
        emails_dict["email_body"] = body
    except:
        emails_dict["email_body"] = None
    
    ###將字典加入list###
    emails_list.append(emails_dict)



In [13]:
#將處理結果轉化為dataframe
emails_df = pd.DataFrame(emails_list)
emails_df

,sender_email,sender_name,recipient_mails,recipient_names,date,emails_body,email_body
0,james_ngola2002@maktoob.com,MR. JAMES NGOLA.,james_ngola2002@maktoob.com,None,None,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-2...
1,bensul2004nng@spinfinder.com,Mr. Ben Suleman,R@M,None,None,URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ..."
2,obong_715@epatra.com,PRINCE OBONG ELEME,obong_715@epatra.com,None,None,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
3,obong_715@epatra.com,PRINCE OBONG ELEME,webmaster@aclweb.org,None,None,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
4,m_abacha03@www.com,Maryam Abacha,m_abacha03@www.com,None,None,I Need Your Assistance.,"Dear sir, \n \nIt is with a heart full of hope..."
5,davidkuta@postmark.net,None,davidkuta@yahoo.com,None,None,Partnership,ATTENTION: ...
6,tunde_dosumu@lycos.com,Barrister tunde dosumu,tunde_dosumu@lycos.com,None,None,Urgent Attention,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)..."
7,william2244drallo@maktoob.com,William Drallo,william2244drallo@maktoob.com,None,None,URGENT BUSINESS PRPOSAL,FROM: WILLIAM DRALLO.\nCONFIDENTIAL TEL: 233-2...
8,abdul_817@rediffmail.com,MR USMAN ABDUL,R@M,None,None,THANK YOU,"CHALLENGE SECURITIES LTD.\nLAGOS, NIGERIA\n\n\..."
9,barrister_td@lycos.com,Tunde Dosumu,barrister_td@lycos.com,None,None,Urgent Assistance,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)..."
